In [40]:
#import 
import pandas as pd
import json
from urllib.request import urlopen 
from datetime import datetime 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as matplot
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import requests
from pandas.plotting import register_matplotlib_converters
import yfinance as yf 

In [41]:
#pull json for cpi
headers = {'Content-type': 'application/json'}
# The series ID CUUR0000SA0L1E is Consumer Price Index - All Urban Consumers

data = json.dumps({"seriesid": ['CUUR0000SA0L1E'],"startyear":"2012", "endyear":"2021"})


p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)

json_data = json.loads(p.text)

In [42]:
#prepare dataframe 
CPI_df = pd.DataFrame(json_data['Results']['series'][0]['data'])

In [43]:
#clean data
CPI_df = CPI_df[['year', 'periodName','value']].set_index('year')

In [44]:
CPI_df['value'] = CPI_df['value'].astype(float)
CPI_df.sort_index(inplace=True)

In [45]:
CPI_df

,periodName,value
year,,
2012,January,227.237
2012,December,231.033
2012,November,231.263
2012,October,231.276
2012,September,230.780
...,...,...
2021,April,273.968
2021,May,275.893
2021,June,278.218


In [46]:
# Pulling gold's price history

gold = yf.Ticker("GC=F")
gold_df = gold.history(start="2016-01-01", end="2021-07-31")
gold_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2016-01-04,1063.400024,1082.500000,1063.199951,1075.099976,143,0,0
2016-01-05,1075.599976,1081.500000,1075.300049,1078.400024,82,0,0
2016-01-06,1081.599976,1093.699951,1081.599976,1091.900024,52,0,0
2016-01-07,1091.599976,1109.400024,1091.599976,1107.699951,122,0,0
2016-01-08,1111.099976,1111.099976,1093.000000,1097.800049,98,0,0


In [37]:
pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23909 sha256=ff91be4ec50d9a28f73eb0850c9256f399e153f9d3ea4e92deb5414fe5448495
  Stored in directory: c:\users\afuen\appdata\local\pip\cache\wheels\fe\87\8b\7ec24486e001d3926537f5f7801f57a74d181be25b11157983
  Created wheel for multitasking: filename=multitasking-0.0.9-py3-none-any.whl size=8368 sha256=28130e6b38f293dd2a2b13f078e8d33216849422ef663dd6a7ca7e6404c665ee
  Stored in directory: c:\users\afuen\appdata\local\pip\cache\wheels\ae\25\47\4d68431a7ec1b6c4b5233365934b74c1d4e665bf5f968d363a
Successfully built yfinance multitasking
Note: you may need to restart the kernel to use updated packages.


In [48]:
# Transform gold data so we can run analysis

gold_df['gold_returns'] = ((gold_df['Close']/gold_df['Open']) - 1) * 100 
gold_returns = gold_df[['gold_returns']]
gold_returns = gold_returns.dropna()
gold_returns.tail()

,gold_returns
Date,
2021-07-26,-0.210821
2021-07-27,-0.232852
2021-07-28,0.000000
2021-07-29,1.361673
2021-07-30,-0.842452


In [49]:
#pull json for marketcap
btc_url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range?vs_currency=usd&from=1451538000&to=1632110400"
btc_data = urlopen(btc_url).read() #Open the API contents
btc_json = json.loads(btc_data) #Transform the contents of our response into a manageable JSON format

In [50]:
btc_json.keys()

dict_keys(['prices', 'market_caps', 'total_volumes'])

In [51]:
df = []
for x in btc_json.keys():
    d = pd.DataFrame(btc_json[x], columns=['date', x])
    d["date"] = pd.to_datetime(d["date"], unit="ms")
    d.set_index("date", inplace=True)
    df.append(d)

In [52]:
df1 = pd.concat(df, axis=1)
df1

,prices,market_caps,total_volumes
date,,,
2016-01-01,434.427000,6.530622e+09,1.338782e+09
2016-01-02,434.326000,6.530689e+09,7.698691e+08
2016-01-03,430.198000,6.470199e+09,9.841860e+08
2016-01-04,433.534000,6.522335e+09,1.021958e+09
2016-01-05,432.115000,6.502629e+09,7.582093e+08
...,...,...,...
2021-09-16,48186.532757,9.067278e+11,3.003782e+10
2021-09-17,47879.473440,9.009905e+11,3.157878e+10
2021-09-18,47228.658216,8.925852e+11,2.773121e+10


In [53]:
# df1["month"] = df1.index.month
# df1["day"] = df1.index.day
# df1.groupby(["month", "day"]).mean()
df1.groupby([df1.index.month, df1.index.day]).mean()

prices   market_caps  total_volumes
date date                                           
1    1      9239.364614  1.655453e+11   1.247299e+10
     2      9519.267228  1.703672e+11   1.070800e+10
     3     10008.244219  1.794067e+11   1.492401e+10
     4     10270.974609  1.841900e+11   3.768596e+10
     5     10406.987763  1.860105e+11   1.955925e+10
...                 ...           ...            ...
12   27    10953.685721  1.956342e+11   1.597680e+10
     28    10660.719176  1.911910e+11   1.922148e+10
     29    10871.194695  1.944676e+11   1.494725e+10
     30    10635.609811  1.908816e+11   1.422590e+10
     31    11138.721085  1.996605e+11   1.501485e+10

[366 rows x 3 columns]

In [56]:
data_combined = pd.concat([CPI_df, df1, gold_returns])
data_combined

,periodName,value,prices,market_caps,total_volumes,gold_returns
2012,January,227.237,NaN,NaN,NaN,NaN
2012,December,231.033,NaN,NaN,NaN,NaN
2012,November,231.263,NaN,NaN,NaN,NaN
2012,October,231.276,NaN,NaN,NaN,NaN
2012,September,230.780,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-07-26 00:00:00,NaN,NaN,NaN,NaN,NaN,-0.210821
2021-07-27 00:00:00,NaN,NaN,NaN,NaN,NaN,-0.232852
2021-07-28 00:00:00,NaN,NaN,NaN,NaN,NaN,0.000000
2021-07-29 00:00:00,NaN,NaN,NaN,NaN,NaN,1.361673


In [58]:
#drop nulls
data_df = data_combined.dropna()

In [59]:
data_df = data_combined.drop_duplicates()